In [1]:
from datasets import load_dataset
raw_datasets = load_dataset("thenlpresearcher/test_data_marathi")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
from evaluate import load

bleu = load("sacrebleu")
chrf = load("chrf")
cometm = load("comet")

2025-11-25 12:38:37.539410: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 12:38:37.603751: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 12:38:40.239261: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Fetching 5 files: 100%|████████████████████| 5/5 [00:00<00:00, 18477.11it/s]
Lig

In [3]:
def compute_scores(preds, ref):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    bleu_score = bleu.compute(predictions=preds, references=ref)["score"]
    chrf_score = chrf.compute(predictions=preds, references=ref)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=ref, char_order=6, word_order=2,  beta=2)["score"]
    comet_score = cometm.compute(sources=src_sentences, predictions=preds, references=ref)["mean_score"]
    return bleu_score, chrf_score, chrf2_score, comet_score

In [4]:
src_sentences = raw_datasets['test']["sent_written"]
ref_gem = raw_datasets['test']["gemini_out"]

## Original

In [5]:
file_name = "shalaka_original_outputs.csv"
mode = "original"

In [6]:
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सवी गायकांनी प्रार्थनेचा उच्चार केल्याने गा...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,जसजसे मशीन विकसित होईल तसतसे आम्ही मागील प्रकल...,"जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [7]:
translations = df['prediction']
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'original'=====
BLEU: 21.72, chrF++: 59.45, chrF2++: 55.38, COMET: 0.7391


## Only Punctuations

In [8]:
file_name = "shalaka_only_punct_outputs.csv"
mode = "only_punct"

In [9]:
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सव साजरा करणार\u093C्याने प्रार्थना केली ते...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन प्राधिकरणांनी निवासी क्षेत्रात आक्षेपार...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,जसजसे मशीन विकसित होईल तसतसे आम्ही मागील प्रकल...,"जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप भ्रामक असू शकते.","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [10]:
translations = df['prediction']
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'only_punct'=====
BLEU: 21.21, chrF++: 58.90, chrF2++: 54.72, COMET: 0.7426


## Without Punctuation

In [11]:
file_name = "shalaka_without_punct_outputs.csv"
mode = "without_punct"

In [12]:
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चा...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या सर्व चारही...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन प्राधिकरणांनी छोट्या व्यवसायांसाठी पर्य...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,"जसजसे मशीन विकसित होते, तसतसे आपण मागील प्रकल्...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप भ्रामक असू शकते.","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [13]:
translations = df['prediction']
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'without_punct'=====
BLEU: 24.66, chrF++: 60.30, chrF2++: 56.56, COMET: 0.7417


## Combined

In [14]:
file_name = "shalaka_combined_outputs.csv"
mode = "combined"

In [15]:
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सव साजरा करणार\u093C्याने प्रार्थनेचा सूर व...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकार\u093C्यांनी छोट्या व्यवसायांसाठी...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,जसजसे मशीन विकसित होईल तसतसे आपण मागील प्रकल्प...,"जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [16]:
translations = df['prediction']
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'combined'=====
BLEU: 25.18, chrF++: 60.49, chrF2++: 56.70, COMET: 0.7436


## Combined LE

In [17]:
file_name = "shalaka_combined_le_outputs.csv"
mode = "combined_le"

In [18]:
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चा...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकार\u093C्यांनी छोट्या व्यवसायांसाठी...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,जसजसे मशीन विकसित होईल तसतसे मागील प्रकल्पांमध...,"जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [19]:
translations = df['prediction']
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'combined_le'=====
BLEU: 24.27, chrF++: 60.61, chrF2++: 56.83, COMET: 0.7443


## Combined LED

In [20]:
file_name = "shalaka_combined_led_outputs.csv"
mode = "combined_led"

In [21]:
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,उत्सव साजरा करणार\u093C्याने प्रार्थनेचा उच्चा...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकार\u093C्यांनी छोट्या व्यवसायांसाठी...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,जसजसे मशीन विकसित होईल तसतसे मागील प्रकल्पांमध...,"जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [22]:
translations = df['prediction']
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'combined_led'=====
BLEU: 24.28, chrF++: 60.21, chrF2++: 56.52, COMET: 0.7433


## GPT-5

In [23]:
file_name = "shalaka_gpt_outputs.csv"
mode = "gpt"

In [24]:
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting the choir raised the volume as the ce...,मंडळातील गायन करताना गायकांचा आवाज वाढवला गेला...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,"सहा महिन्यांचा बछडा तपासणीसाठी सादर केला गेला,...",जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन प्राधिकरणांनी अशा लघु व्यवसायांसाठी पर्...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,As the machine develops the forms we use to re...,"मशीन जसे स्वरूप विकसित करते, तसेच आपण भूतपूर्व...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","जसे म्हटले, पहिले छाप चुकीचे ठरू शकतात.","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [25]:
translations = df['prediction']
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'gpt'=====
BLEU: 18.69, chrF++: 52.50, chrF2++: 48.82, COMET: 0.7420


## BERT

In [26]:
file_name = "approach1_bert_outputs.csv"
mode = "bert"

In [27]:
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,Chanting: the choir raised the volume as the c...,जपः गायकवृंदाने आवाज वाढवला आणि उत्सवी व्यक्ती...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,"As the machine develops, the forms we use to r...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned,, first impressions can be mislea...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [28]:
translations = df['prediction']
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'bert'=====
BLEU: 23.84, chrF++: 60.02, chrF2++: 56.11, COMET: 0.7595


## MPNet

In [29]:
file_name = "approach1_mpnet_outputs.csv"
mode = "mpnet"

In [30]:
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,"Chanting, the choir raised the volume as the c...",उत्सवी व्यक्तीने प्रार्थनेचा उच्चार करत असताना...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-- old calf was submitted for exami...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,"As the machine develops, the forms we use to r...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned,, first impressions can be mislea...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [31]:
translations = df['prediction']
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 'mpnet'=====
BLEU: 25.12, chrF++: 60.56, chrF2++: 56.79, COMET: 0.7597


## T5

In [32]:
file_name = "approach1_t5_outputs.csv"
mode = "t5"

In [33]:
df = pd.read_csv(file_name)
df.head()

,src,prediction,gt,gemini,cfilt
0,"Chanting, the choir raised the volume as the c...",उत्सवी व्यक्तीने प्रार्थनेचा उच्चार करत असताना...,जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार क...,"धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या ...","उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, ..."
1,A six-month-old calf was submitted for examina...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...,तपासणीसाठी आणलेल्या सहा महिन्यांच्या एका वासरा...,जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पाया...
2,Planning authorities should provide alternativ...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...,नियोजन प्राधिकरणांनी लहान व्यवसायांसाठी पर्याय...,नियोजन अधिकाऱ्यांनी छोट्या व्यवसायांसाठी पर्या...
3,"As the machine develops, the forms we use to r...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजसे यंत्र विकसित होईल तसतसे, मागील प्रकल्पां...","जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्प...","जसजसे मशीन विकसित होईल, तसतसे मागील प्रकल्पांम..."
4,"As mentioned, first impressions can be mislead...","नमूद केल्याप्रमाणे, पहिली छाप दिशाभूल करणारी अ...","आधी नमूद केल्याप्रमाणे, छाप दिशाभूल करणारी असू...","आधी सांगितल्याप्रमाणे, पहिली छाप फसवी असू शकते.","प्रथम नमूद केल्याप्रमाणे, छाप दिशाभूल करणारे अ..."


In [34]:
translations = df['prediction']
bleu_score, chrf_score, chrf2_score, comet_score = compute_scores(translations, ref_gem)

print(f"\n===== FINAL METRICS FOR '{mode}'=====")
print(f"BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}, COMET: {comet_score:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



===== FINAL METRICS FOR 't5'=====
BLEU: 24.74, chrF++: 60.68, chrF2++: 56.92, COMET: 0.7586
